In [117]:
import pandas as pd
from fuzzywuzzy import (fuzz, process)
import random
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [120]:
data_new=pd.read_csv('/home/yash/Downloads/Deduplication Problem - Sample Dataset.csv')

In [121]:
data_new.head(10)

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM
5,BLAND JR,21/02/62,F,WILLIAM
6,BLAND JR,21/02/62,F,WILLIAM
7,BLAND JR,21/02/62,F,WILLIAM
8,SHAFFER JR,21/02/62,F,WILLIAM
9,SHAFFER JR,21/02/62,F,WILLIAM


In [122]:
name=data_new.ln.apply(lambda x:x.split(' '))
new_name=[]
for i in name:
    if len(i)==1:
        i.append('Nan')
    new_name.append(i[1])
name_f=data_new.fn.apply(lambda x:x.split(' '))
new_name_f=[]
for i in name_f:
    if len(i)==1:
        i.append('Nan')
    new_name_f.append(i[1])    
    

In [123]:
data_new['FIRST']=data_new['fn'].apply(lambda x:x.split(' ')[0])
data_new['LAST']=data_new.ln.apply(lambda x:x.split(' ')[0])
data_new['SUFFIX']=new_name
data_new['GENDER']=data_new['gn'].values
data_new['DOB']=data_new['dob'].values
data_new['MIDDLE']=new_name_f
df=data_new[['FIRST','MIDDLE','LAST','SUFFIX','DOB','GENDER']]

In [124]:
df.head()

,FIRST,MIDDLE,LAST,SUFFIX,DOB,GENDER
0,WILLIAM,Nan,SMITH,JR,01/03/68,F
1,WILLIAM,Nan,ROTHMEYER,JR,01/03/68,F
2,WILLIAM,Nan,ASBY,JR,01/03/68,F
3,WILLIAM,Nan,SALTER,JR,01/03/68,F
4,WILLIAM,Nan,SALTER,JR,01/03/68,F


In [125]:
df['concat'] = [' '.join(row) for row in df.values]
df['concat'] = [' '.join(s.split()) for s in df['concat']]

# Randomly select a row to compare against all others
test_s = random.choice(df['concat'].values)
possible_values = set(df['concat'].values).difference({test_s})

# fuzzywuzzy.fuzz.ratio is a nice wrapper for difflib.SequenceMatcher
match_proba = {s: fuzz.ratio(s, test_s) for s in possible_values}

print(test_s, '\n\n', sorted(match_proba.items(), key=lambda x: x[1], reverse=True)[:5])

WILLIAM Nan BLAND JR 08/06/54 F 

 [('WILLIAM Nan BLAND JR 25/10/53 F', 87), ('WILLIAM Nan BLAND JR 21/02/62 F', 84), ('WILLIAM Nan BLAND Nan 21/02/62 F', 76), ('WILLIAM Nan BLAND III 21/02/62 F', 76), ('WILLIAM Nan ASBY JR 01/03/68 F', 75)]


In [129]:
df['FULLNAME'] = [{'FIRST': first, 'MIDDLE': middle, 'LAST': last,'SUFFIX':suffix}
                  for first, middle, last,suffix in df[['FIRST', 'MIDDLE', 'LAST','SUFFIX']].values]

def create_custom_dict_column(columns, column_name='custom', frame=df):
   
    df_as_dict = frame[columns].to_dict()
   
    df_as_dict = {col: rows for col, rows in df_as_dict.items() if col in columns}
    # Get all indices
    idxs = {idx for col in df_as_dict for idx in df_as_dict[col]}
    
    row_data = {idx: ' '.join(df_as_dict[col][idx] for col in columns) for idx in idxs}
    df_to_join = pd.DataFrame(data=list(row_data.values()),
                              columns=[column_name], index=list(row_data.keys()))
    frame = frame.join(df_to_join, how='left')
    return frame

In [130]:
test_name = random.choice(df['FULLNAME'].values)

def fuzz_ratio_dict_wrapper(a: dict, b: dict):
    keys = set(a.keys()) & set(b.keys())
    fuzz_ratios = {k: fuzz.ratio(a[k], b[k])
                  for k in keys}
    return sum(fuzz_ratios.values())

print(test_name, '\n\n', sorted([(name, fuzz_ratio_dict_wrapper(test_name, name)) 
                                 for name in df['FULLNAME'].values if name != test_name],
                                 key=lambda x: x[1], reverse=True)[:5])

{'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'FAGEN', 'SUFFIX': 'JR'} 

 [({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'MICHAELSON', 'SUFFIX': 'JR'}, 340), ({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'MICHAELSON', 'SUFFIX': 'JR'}, 340), ({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'LARSON', 'SUFFIX': 'JR'}, 336), ({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'LARSON', 'SUFFIX': 'JR'}, 336), ({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'MELVIN', 'SUFFIX': 'JR'}, 336)]


In [131]:
def reduce_dicts(dicts, keys_to_include):
    '''Filters dictionaries to include only a passed set of keys
    
    Parameters:
        - dicts: iterable of dict objects
        - keys_to_include: iterable of keys to keep'''
    return [{k: v for k, v in d.items() if k in keys_to_include} for d in dicts]
    
def name_similarity(a: dict, b: dict, which_names={'FIRST', 'MIDDLE', 'LAST'}):
    '''Intended originally for comparing names, this function could really be
    extended to comparisons of any sequences'''
    # Reduce the name dictionaries to only the ones in `which_names`
    a, b = reduce_dicts([a, b], which_names)

    a_set = set(a.values())
    b_set = set(b.values())
    
    def jaccard_index(A: set, B: set):
        '''Intersection-over-union measure'''
        return len((A & B))/len((A | B))
    
    def length_similarity(a: dict, b: dict):
        '''Compares lengths of string values in passed dictionaries. 
        The smaller the value returned, the more similar the values 
        are from a length point of view'''
        matching_keys = set(a.keys()) & set(b.keys())
        len_diffs = {k: abs(len(a[k]) - len(b[k])) for k in matching_keys}
        return sum(len_diffs.values())
    
    def element_overlap(A: set, B: set, exp=1):
        '''Set intersection with the ability to reward sets
        sharing many elements (using `exp`)'''
        return len(A & B) ** exp
    
    fuzzratio = fuzz_ratio_dict_wrapper(a, b)
    jaccard = jaccard_index(a_set, b_set)
    length_sim = length_similarity(a, b)
    element_overlap_value = element_overlap(a_set, b_set, exp=2)
    
    return fuzzratio + jaccard + element_overlap_value - length_sim

# Sample input and output

In [132]:
test_name = random.choice(df['FULLNAME'].values)

print(test_name, '\n', sorted([(name, name_similarity(test_name, name))
                         for name in df['FULLNAME'].values if name != test_name],
                         key=lambda x: x[1], reverse=True)[:5])


{'FIRST': 'THOMAS', 'MIDDLE': 'Nan', 'LAST': 'DUNCAN', 'SUFFIX': 'JR'} 
 [({'FIRST': 'ROY', 'MIDDLE': 'Nan', 'LAST': 'DUNCAN', 'SUFFIX': 'JR'}, 223.5), ({'FIRST': 'JOHN', 'MIDDLE': 'Nan', 'LAST': 'CAIN', 'SUFFIX': 'JR'}, 177.2), ({'FIRST': 'RONALD', 'MIDDLE': 'Nan', 'LAST': 'CLARK', 'SUFFIX': 'Nan'}, 169.2), ({'FIRST': 'DONALD', 'MIDDLE': 'Nan', 'LAST': 'CLARK', 'SUFFIX': 'Nan'}, 169.2), ({'FIRST': 'HAROLD', 'MIDDLE': 'Nan', 'LAST': 'CHAVAS', 'SUFFIX': 'JR'}, 167.2)]
